In [1]:
import os
import glob
import pandas as pd
from pathlib import Path

In [3]:
root_dir = "/mnt/nfs/stricon_data/stricon_resting_state/derivatives_new"
root = Path(root_dir)
subj_test_list = []
subjects = sorted([p for p in root.iterdir() if p.is_dir() and p.name.startswith("sub-")])
for subj in subjects:
    subj_test_list.append(subj)
    
subj_test_list = subj_test_list[:3]
subj_test_list

[PosixPath('/mnt/nfs/stricon_data/stricon_resting_state/derivatives_new/sub-1'),
 PosixPath('/mnt/nfs/stricon_data/stricon_resting_state/derivatives_new/sub-10'),
 PosixPath('/mnt/nfs/stricon_data/stricon_resting_state/derivatives_new/sub-100')]